In [11]:
#openai
from openai import OpenAI
from pydantic import BaseModel
import os
from dotenv import load_dotenv

load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_KEY = os.getenv("SUPABASE_SERVICE_KEY")

#openai
client = OpenAI(api_key = OPENAI_API_KEY)

#supabase
from supabase import create_client, Client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

In [12]:
#Basic Fact Agent

class basicFactResponse(BaseModel):
    basic_facts: list[str]  # 여러 문장 형태로 반환

def extract_basic_facts(user_input: str) -> list[str]:
    messages = [
        {
            "role": "system",
            "content": """
**역할**
너는 클라이언트가 작성한 사건 설명을 읽고, 그로부터 법률적 분석을 위한 **기초사실(basic facts)** 을 항목별로 추출하는 역할을 맡은 변호사야.

**지침**
- 사용자의 긴 설명에서 **사건의 발생 경위, 계약 내용, 일자, 장소, 쟁점 관련 사실**들을 논리적으로 정리해.
- 각 기초 사실은 문장 하나로 표현하고, **판례 형식의 표현 스타일**을 따를 것.
- 시간순으로 서술하되, 논리적 맥락이 흐름 있게 이어지도록 구성할 것.
- 의견, 감정, 해석은 제거하고 **객관적 사실**만 기술할 것.

**출력 형식 예시**
1. 원고는 2021년 3월 1일, 피고 소유의 서울시 강남구 소재 아파트를 전세보증금 1억 원에 임차하였다.  
2. 계약기간은 2년으로 2023년 2월 28일까지로 정해졌다.  
3. 원고는 계약 종료일에 맞추어 이사를 완료하였고, 보증금 반환을 요청하였다.  
4. 피고는 자금 사정을 이유로 보증금 반환을 거절하였다.  
5. 원고는 보증금 반환 요청을 내용증명으로 2023년 3월 5일에 통지하였다.

**답변은 위와 같이 번호 목록 형식으로 작성해줘.**
"""
        },
        {
            "role": "user",
            "content": user_input
        }
    ]

    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=basicFactResponse,
        messages=messages
    )

    return completion.choices[0].message.parsed.basic_facts

In [13]:
#쟁점 분석 에이전트

class LegalIssueResponse(BaseModel):
    legal_issue: str

def generate_legal_issue(description):
    messages = [
        {
            "role": "system",
            "content": """
            **역할**
            너는 민사, 형사, 행정 사건에서 발생하는 복수의 쟁점을 분석하는 법률 전문가야.  
            아래 ##사건 설명##을 보고, 관련된 **모든 법적 쟁점**을 빠짐없이, 명확하고 전문적으로 도출해줘.

            **지침**
            - 반드시 한 줄 요약이 아닌, 다양한 쟁점들을 항목별로 나열해줘.
            - 각 쟁점은 구체적인 법률적 표현을 포함하여 명확하게 작성해.
            - 쟁점은 실제 민사소송에서 다뤄질 수 있는 수준으로, 책임, 권리, 절차, 지연손해금 등도 빠짐없이 포함해.
            - 출력은 아래 형식을 반드시 지켜:
            <1. ~에 관한 쟁점>  
            <2. ~에 관한 쟁점>  
            …

            **답변 예시**
            ##사건 설명##
            임대차 계약이 종료되었으나 집주인이 보증금을 반환하지 않고 있으며, 임차인은 이사비용으로 인해 은행 대출까지 받은 상태입니다. 집주인은 반환일정도 제시하지 않고 있으며 연락도 회피하고 있습니다.

            ##법적 쟁점##
            1. 임대차 계약 종료 후 임대인의 보증금 반환의무 불이행에 대한 책임  
            2. 임차인의 보증금 반환청구권 행사와 이에 따른 법적 절차 (내용증명, 소송 제기 등)  
            3. 임대인의 연락 회피 및 지연에 따른 채무불이행 인정 여부  
            4. 지연손해금 청구의 법적 근거와 적용 범위  
            5. 임차인의 대출 발생에 따른 추가 손해에 대한 배상 가능성  
            6. 반환불이행 시 강제집행을 위한 보전처분(가압류 등)의 필요성  
            7. 주택임대차보호법에 따른 임차인 보호 조항 적용 여부  
            8. 소멸시효 및 반환청구 시의 입증 책임 문제

            ---------------------------
            답변은 위 예시처럼 숫자 목록 형식으로 출력해줘.
            """
        },
        {
            "role": "user",
            "content": description
        }
    ]

    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=LegalIssueResponse,
        messages=messages
    )
    return completion.choices[0].message.parsed.legal_issue

In [14]:
import json

#판례 검색용 질의 생성 에이전트

def generate_precedent_queries(
    legal_issue: str,
    basic_facts: list[str],
    case_categories: list[str]
) -> list[str]:
    system_prompt = """
너는 변호사이자 리걸 엔지니어로서 사용자가 제공한 '법적 쟁점', '기초 사실들', '사건 분야(민사, 형사, 행정)'을 분석하고,
그에 기반해 판례 검색 시스템에서 사용할 수 있는 **다양한 문장 형태의 판례 검색 질의문들을 생성**하는 역할이야.

**생성 원칙**
- 각 문장은 검색 시스템의 질의로 사용할 수 있는 자연어 문장이어야 한다.
- 반드시 현실의 사건을 검색하듯, **검색 키워드가 포함된 문장 형태**로 쓸 것
- 질의마다 관점(행위자, 쟁점, 청구 취지 등)을 달리해 **다양한 검색 경로**를 제시할 것
- 사건의 분야(민사/형사/행정)를 고려해 해당 법 분야 특성에 맞는 키워드와 문맥을 반영할 것

**출력 예시**
[
  "보증금 반환을 거절하는 임대인에 대한 임차인의 반환청구 관련 판례",
  "임대차 계약 종료 후 보증금을 반환하지 않은 경우의 분쟁 사례",
  "임대차 계약 만료 후 집주인의 반환 지연에 대한 손해배상 관련 판례"
]

**출력 형식**
- 각 문장 끝은 '~ 판례'로 마무리
"""

    facts_summary = "\n".join(f"- {fact}" for fact in basic_facts[:10])
    category_summary = ", ".join(case_categories)

    user_prompt = f"""
## 사건 분야 ##
{category_summary}

## 법적 쟁점 ##
{legal_issue}

## 기초 사실 ##
{facts_summary}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt.strip()}
    ]

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )

    try:
        return json.loads(response.choices[0].message.content.strip())
    except:
        return [
            line.strip("-• ").strip()
            for line in response.choices[0].message.content.strip().splitlines()
            if line.strip()
        ]

In [15]:
# 사건 분야 분류 에이전트

from pydantic import BaseModel
from typing import List
from openai import OpenAI

# ✅ OpenAI 클라이언트 초기화 (환경변수에 OPENAI_API_KEY가 있어야 함)
import os
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ✅ Pydantic 모델 정의
class CaseDomainResponse(BaseModel):
    domains: List[str]

# ✅ 분류 함수 정의
def classify_legal_domains(client_input: str) -> List[str]:
    system_prompt = """
**역할**
너는 클라이언트가 서술한 사건 설명을 읽고, 이 사건이 어느 법률 영역(민사, 형사, 행정)에 해당하는지 판단하는 법률 전문가야.

**판단 기준**
- 민사: 개인 간의 재산, 계약, 손해배상, 임대차, 부당이득, 불법행위 등
- 형사: 범죄 행위 (절도, 폭행, 사기 등)에 따른 형사처벌 또는 형사소송
- 행정: 공무원 징계, 허가취소, 세금, 행정청의 처분에 대한 불복 등 행정기관 상대 사건

**출력 형식**
["민사"], ["형사"], ["민사", "형사"]처럼 리스트 형태로 판단 결과를 제공해줘. 절대 다른 단어나 설명 없이!

예시:
- “회사에서 해고당했는데 부당해고라고 생각해요.” → [“민사”]
- “도로교통법 위반으로 벌금형을 받았는데 억울해요.” → [“형사”]
- “공무원인데 정직 처분을 받았습니다.” → [“행정”]
- “상해죄로 고소도 당했지만, 상대방에게 치료비 청구도 하고 싶어요.” → [“형사”, “민사”]
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": client_input}
    ]

    # ✅ 구조화된 응답 파싱
    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=messages,
        response_format=CaseDomainResponse
    )

    return response.choices[0].message.parsed.domains

In [16]:
#관련 조문 추천 에이전트

from pydantic import BaseModel
from typing import List
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

class RelevantLawListResponse(BaseModel):
    relevant_laws: List[str]

def recommend_relevant_laws(
    legal_issue: str,
    facts: List[str],
    case_categories: List[str]
) -> List[str]:
    system_prompt = """
**역할**
너는 법률 전문가로서, 사용자가 제공한 '법적 쟁점', '기초 사실들', '사건 분야'를 종합 분석하여 **정확하고 관련성 높은 법령 조항**을 **가능한 한 많이** 추천하는 역할을 맡았어.

**지침**
- 반드시 해당 사건의 쟁점과 사실에 관련된 조문만 추천할 것.
- 각 조문은 아래 형식으로 작성:
  "<법령명> 제X조 (조문 제목)"
- 한 조문당 한 줄씩 리스트로 작성하고, 관련이 낮거나 유사 조항은 제외할 것.
- 최대한 다양한 법령(민법, 형법, 약관법 등 포함)을 반영해도 좋지만, 쟁점과의 관련성이 가장 중요함.

**예시 출력**
[
  "민법 제618조 (임대차의 정의)",
  "민법 제623조 (임대인의 의무)",
  "민법 제750조 (불법행위)",
  "주택임대차보호법 제3조 (대항력 등)",
  "주택임대차보호법 제4조 (계약갱신과 보증금 반환)"
]
"""

    facts_summary = "\n".join(f"- {fact}" for fact in facts[:10])
    domain_text = ", ".join(case_categories)

    user_prompt = f"""
## 사건 분야 ##
{domain_text}

## 법적 쟁점 ##
{legal_issue}

## 기초 사실 ##
{facts_summary}
"""

    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=RelevantLawListResponse,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.strip()}
        ]
    )

    return response.choices[0].message.parsed.relevant_laws

In [17]:
# #판례 검색 에이전트

# # search_similar_precedents_from_supabase.py
# import os
# from typing import List, Tuple, Set, Dict
# from collections import Counter
# from dotenv import load_dotenv
# from supabase import create_client, Client
# from openai import OpenAI

# load_dotenv()
# supabase: Client = create_client(os.getenv("SUPABASE_URL"),
#                                  os.getenv("SUPABASE_SERVICE_KEY"))
# client           = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


# def search_similar_precedents_from_supabase(
#     precedent_queries: List[str],
#     legal_issue: str,
#     basic_facts: List[str],
#     legal_domains: List[str] | None = None,
# ) -> Tuple[str, List[Dict]]:
#     """RPC 함수를 이용해 판례 검색 → 요약"""
#     matched: List[Dict] = []

#     for keyword in precedent_queries:
#         q = keyword.strip()
#         if not q:
#             continue

#         for dom in (legal_domains or [None]):   # None → casetype 전체
#             try:
#                 resp = supabase.rpc(
#                     "rpc_search_precedents",
#                     {"q": q, "p_domain": dom, "p_limit": 15},
#                 ).execute()
#                 matched.extend(resp.data)
#             except Exception as e:
#                 print(f"❗ Supabase RPC 오류: {q} / {e}")

#     # ── 중복 제거 ──────────────────────────
#     seen: Set[str] = set()
#     unique_cases: List[Dict] = []
#     for case in matched:
#         if case["caseno"] not in seen:
#             seen.add(case["caseno"])
#             unique_cases.append(case)

#     if not unique_cases:
#         return ("🔍 유사 판례를 찾을 수 없습니다.", [])

#     # ── 요약 프롬프트 ───────────────────────
#     prompt = f"""
# 다음 사건의 쟁점·기초 사실 및 유사 판례 목록이야.
# 유사점·차이점, 판결 경향을 6줄 내로 요약해줘.

# ## 쟁점
# {legal_issue}

# ## 사실
# {chr(10).join('- ' + f for f in basic_facts)}

# ## 판례
# {chr(10).join('- ' + c['caseno'] + ' / ' + (c.get('casenm') or '') for c in unique_cases)}
# """.strip()

#     completion = client.chat.completions.create(
#         model="gpt-4o",
#         temperature=0.3,
#         messages=[
#             {"role": "system", "content": "너는 한국 판례 분석 전문가야."},
#             {"role": "user", "content": prompt},
#         ],
#     )
#     return completion.choices[0].message.content.strip(), unique_cases

In [50]:
# ──────────────────────────────────────────────
# Supabase 기반 판례 검색 에이전트 (vector-based re-ranking)
# ──────────────────────────────────────────────
"""
Prerequisite
------------
1. Supabase Postgres에 pgvector 확장 설치
   CREATE EXTENSION IF NOT EXISTS vector;
2. precedents_full 테이블에 다음 벡터 컬럼 존재
   - bsisfacts_vector    vector(768)
   - courtdcss_vector    vector(768)
   - relatelaword_vector vector(768)
3. Python 패키지: openai, supabase-py, numpy, python-dotenv
"""
from __future__ import annotations

import os, json, numpy as np
from typing import List, Tuple
from functools import lru_cache
from dotenv import load_dotenv
from supabase import create_client, Client
from openai import OpenAI

# ──────────────────── 환경 설정 ────────────────────
load_dotenv()
SUPABASE_URL         = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_KEY = os.getenv("SUPABASE_SERVICE_KEY")
OPENAI_API_KEY       = os.getenv("OPENAI_API_KEY")

supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)
oai              = OpenAI(api_key=OPENAI_API_KEY)

# ──────────────────── util: any → str ───────────────
def _to_text(x) -> str:
    """
    embed() 에 안전하게 넘길 수 있도록 타입을 문자열로 정규화
    • list/tuple  → 줄바꿈 join
    • dict       → pretty JSON
    • None       → ""
    • 기타       → str(x)
    """
    if x is None:
        return ""
    if isinstance(x, (list, tuple)):
        return "\n".join(map(str, x))
    if isinstance(x, dict):
        return json.dumps(x, ensure_ascii=False, indent=2)
    return str(x)

# ──────────────────── util: 임베딩 ──────────────────
@lru_cache(maxsize=1024)
def embed(text: str, model: str = "text-embedding-3-small") -> List[float]:
    """OpenAI 임베딩 호출 결과를 LRU 캐시"""
    resp = oai.embeddings.create(model=model, input=text)
    return resp.data[0].embedding    # 1536-D (3-small 기준)

# ──────────────────── main ─────────────────────────
def search_similar_precedents_from_supabase(
    *,                               # keyword-only
    basic_facts:  str | list | dict | None = None,
    legal_issue:  str | list | dict | None = None,
    related_laws: str | list | dict | None = None,
    # LangGraph alias
    precedent_queries: str | list | None = None,
    legal_domains:     str | None = None,
    case_type:         str | None = None,
    # search params
    top_k:   int   = 10,
    w_facts: float = 0.4,
    w_issue: float = 0.4,
    w_law:   float = 0.2,
) -> Tuple[str, List[dict]]:
    """pgvector + GPT-4o 요약 기반 유사 판례 검색"""

    # 0️⃣ alias 매핑 -------------------------------------------------
    if case_type is None and legal_domains is not None:
        case_type = legal_domains
    case_type = case_type or "civil"          # 기본값

    # 👉 리스트·튜플이면 첫 번째 원소(또는 join)만 사용
    if isinstance(case_type, (list, tuple)):
        case_type = case_type[0] if case_type else "civil"
    case_type = str(case_type)                # 방어적 캐스팅

    # 🆕 영문 → 국문 매핑
    CASE_TYPE_MAP = {
        "civil"   : "민사",
        "criminal": "형사",
        "admin"   : "행정",
    }
    case_type = CASE_TYPE_MAP.get(case_type, case_type)   # 변환

    if basic_facts is None and precedent_queries is not None:
        basic_facts = precedent_queries        # 리스트일 수도 있음

    # 1️⃣ 문자열 정규화 ---------------------------------------------
    basic_facts  = _to_text(basic_facts)
    legal_issue  = _to_text(legal_issue)
    related_laws = _to_text(related_laws)

    # 2️⃣ 임베딩 ------------------------------------------------------
    fact_vec  = embed(basic_facts)
    issue_vec = embed(legal_issue)
    law_vec   = embed(related_laws)

    # 3️⃣ Supabase RPC 호출 -----------------------------------------
    payload = {
        "case_type": case_type,
        "fact_vec":  fact_vec,
        "issue_vec": issue_vec,
        "law_vec":   law_vec,
        "w_f":       w_facts,
        "w_i":       w_issue,
        "w_l":       w_law,
        "k":         top_k,
    }
    res = supabase.rpc("top_k_precedents", payload).execute()
    matches: List[dict] = res.data or []
    if not matches:
        return "🔍 유사한 판례를 찾을 수 없습니다.", []

    # 4️⃣ GPT-4o 판례 요약 ------------------------------------------
    cases_list = "\n".join(
        f"- 사건번호: {c['caseno']} / 사건명: {c.get('casenm','')}"
        for c in matches
    )
    summary_prompt = f"""
당신은 판례 분석 전문가입니다. 아래 사건의 사실관계·쟁점에 유사한 판례 요약을 작성하세요.

## 기초 사실
{basic_facts}

## 법적 쟁점
{legal_issue}

## 관련 조문
{related_laws}

## 유사 판례 목록
{cases_list}
"""
    chat = oai.chat.completions.create(
        model="gpt-4o",
        temperature=0.3,
        messages=[
            {"role": "system", "content": "당신은 법률 판례 요약 전문가입니다."},
            {"role": "user",    "content": summary_prompt.strip()},
        ],
    )
    summary_text = chat.choices[0].message.content.strip()
    return summary_text, matches

In [19]:
#법원 판단 시뮬레이션 에이전트

class LegalJudgmentResponse(BaseModel):
    judgment_summary: str

def simulate_judgment(facts, precedents_summary, law_articles, case_type):
    messages = [
        {
            "role": "system",
            "content": f"""
**역할**
너는 실제 법원 판결문을 작성하는 판사야. 사건의 사실관계, 법적 쟁점, 사건 분야, 적용 법령, 그리고 유사 판례들을 모두 고려하여 최종적인 법원의 판단을 내리는 역할이야.

**목표**
주어진 정보만으로 민사/형사/행정 사건에 대해 판결문 중 '판단 이유' 부분을 작성하듯, 법원이 내릴 법적 판단을 구체적이고 체계적이며 매우 전문적인 논증으로 정리해.

**출력 기준**
- 단 하나의 결론이 아니라, 논증 전개를 포함해 실제 판결문처럼 길고 깊이 있게 작성해
- 법적 쟁점별로 판단 근거를 구조적으로 제시해
- 유사 판례와 법 조항의 근거를 결합해 논리적으로 설명하고, 왜 해당 사건에도 동일하게 판단해야 하는지 밝혀줘
- 판례와의 유사성과 차이점, 법 조항의 해석 적용 과정도 자세히 설명해
- 끝부분에 결론을 명시하되, 단순히 "책임이 인정된다" 수준이 아니라 왜 그렇게 판단하는지를 정리해
- 문장 하나하나가 실제 법관의 언어처럼 설득력 있고 신중하게 구성되어야 함
- 형식은 반드시 '예상 판단 요지:' 없이 **판결문 판단 부분처럼 자연스럽게 시작**할 것
"""
        },
        {
            "role": "user",
            "content": f"""
##사건 분야##
{case_type}

##사실관계##
{facts}

##유사 판례 요약##
{precedents_summary}

##적용 법령##
{law_articles}
"""
        }
    ]

    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=LegalJudgmentResponse,
        messages=messages
    )
    return completion.choices[0].message.parsed.judgment_summary


In [20]:
#양형 예측 에이전트

class SentencePredictionResponse(BaseModel):
    predicted_sentence: str

def predict_sentence(facts, law_articles, precedent_summary):
    messages = [
        {
            "role": "system",
            "content": """
**역할**
너는 형사 사건에 대해 선고될 수 있는 **가장 현실적이고 개연성 높은 형량**을 예측하는 판사 역할이야.

**입력 정보**는 아래와 같아:
- 사건 분야 (형사)
- 기초 사실 (basic facts)
- 법적 쟁점 (legal issue)
- 판례 검색용 질의 (precedent queries)
- 사건 분야 분류 결과 (분류된 카테고리)
- 관련 판례 요약 (precedent summary)
- 적용 법령 목록 (relevant laws)
- 법원의 예상 판단 요지 (legal judgment)

너는 이 모든 정보를 통합적으로 고려해서, **선고 형량**을 구체적으로 예측해야 해.

**형식 요건**
- 반드시 형식은 아래와 같아야 함:
    - 징역 ○년 ○월
    - 징역 ○년 ○월, 집행유예 ○년
    - 벌금 ○○만원
- 형사소송 관행에 부합하도록 실제 선고문처럼 **사실관계의 중대성**, **반성 여부**, **피해 회복**, **전과 유무**, **양형기준**, **참작 사유** 등을 종합 판단해.
- 반드시 실형/집행유예/벌금 여부와 기간 또는 액수를 정확히 제시할 것
- 변호사나 법조인에게 제공할 수준의 실무적, 논리적 타당성을 갖출 것
"""
        },
        {
            "role": "user",
            "content": f"""##사실관계##\n{facts}\n\n##적용 법령##\n{law_articles}\n\n##유사 판례 요약##\n{precedent_summary}"""
        }
    ]

    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=SentencePredictionResponse,
        messages=messages
    )
    return completion.choices[0].message.parsed.predicted_sentence

In [21]:
!pip install langfuse
!pip install langgraph

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [22]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_handler = CallbackHandler(
  secret_key="sk-lf-bc52adc2-5b9e-4516-b4d6-3c98ac4cee1e",
  public_key="pk-lf-e8844606-a8dd-4b87-9976-c2d151c27983",
  host="https://us.cloud.langfuse.com"
)

In [23]:
# Langfuse CallbackHandler for Langgraph
from __future__ import annotations        # ⇒ list[str] 타입 힌트 지원
from typing import TypedDict, Optional
from langgraph.graph import StateGraph

class LegalCaseState(TypedDict):
    # ───── 읽기 전용 ─────
    user_input: str                       # 최초 사용자 질문

    # ───── 각 노드에서 업데이트할 필드 ─────
    legal_issue: Optional[str]            # ✦ 이슈 추출 노드
    precedent_queries: Optional[list[str]]# ✦ 판례 검색 쿼리 생성 노드
    precedent_summary: Optional[str]      # ✦ 판례 요약 노드
    law_recommendation: Optional[str]     # ✦ 관련 조문 추천 노드
    legal_judgment_prediction: Optional[str]  # ✦ 판결 결과 예측 노드
    sentence_prediction: Optional[str]    # ✦ 형량 예측 노드
    basic_facts: Optional[list[str]]      # ✦ 사실관계 추출 노드
    case_categories: Optional[list[str]]  # ✦ 사건 분류 노드
    final_answer: Optional[str]           # ✦ 최종 답변 조립 노드

workflow = StateGraph(LegalCaseState)

In [24]:
from typing import TypedDict, Optional

def classify_legal_domains_node(state: LegalCaseState) -> LegalCaseState:
    categories = classify_legal_domains(state["user_input"])
    return {"case_categories": categories}                 # ✅

def extract_basic_facts_node(state: LegalCaseState) -> LegalCaseState:
    facts = extract_basic_facts(state["user_input"])
    return {"basic_facts": facts}                          # ✅

def extract_legal_issue_node(state: LegalCaseState) -> LegalCaseState:
    issue = generate_legal_issue(state["user_input"])
    return {"legal_issue": issue}                          # ✅

def generate_precedent_queries_node(state: LegalCaseState) -> LegalCaseState:
    queries = generate_precedent_queries(
        legal_issue=state["legal_issue"],
        basic_facts=state["basic_facts"],
        case_categories=state["case_categories"]
    )
    return {"precedent_queries": queries}                  # ✅

def summarize_precedents_node(state: LegalCaseState) -> LegalCaseState:
    summary, matches = search_similar_precedents_from_supabase(
        precedent_queries=state["precedent_queries"],
        legal_issue=state["legal_issue"],
        basic_facts=state["basic_facts"],
        legal_domains=state["case_categories"]
    )
    return {
        "precedent_summary": summary,
        "precedent_matches": matches                       
    }

def recommend_law_node(state: LegalCaseState):
    laws = recommend_relevant_laws(
        legal_issue     = state["legal_issue"],
        facts           = state["basic_facts"],
        case_categories = state["case_categories"],
    )
    return {"law_recommendation": laws}

def simulate_judgment_node(state: LegalCaseState) -> LegalCaseState:
    judgment = simulate_judgment(
        facts=state["basic_facts"],
        precedents_summary=state["precedent_summary"],
        law_articles=state["law_recommendation"],
        case_type=state["case_type"]
    )
    return {"legal_judgment_prediction": judgment}

def predict_sentence_node(state: LegalCaseState) -> LegalCaseState:
    sentence = predict_sentence(
        facts=state["basic_facts"],
        law_articles=state["law_recommendation"],
        precedent_summary=state["precedent_summary"]
    )
    return {"sentence_prediction": sentence}

def generate_final_answer_node(state: LegalCaseState) -> LegalCaseState:
    parts = [
        f"✅ 사건 분야: {', '.join(state['case_categories'])}" if state.get("case_categories") else "",
        f"🔎 기초 사실: {'; '.join(state['basic_facts'])}"    if state.get("basic_facts") else "",
        f"⚖️ 법적 쟁점: {state['legal_issue']}"             if state.get("legal_issue") else "",
        f"📖 적용 법령: {state['law_recommendation']}"       if state.get("law_recommendation") else "",
        f"📚 유사 판례 요약: {state['precedent_summary']}"  if state.get("precedent_summary") else "",
        f"🧑‍⚖️ 예상 판결 요지: {state['legal_judgment_prediction']}" if state.get("legal_judgment_prediction") else "",
    ]
    if state.get("sentence_prediction"):
        parts.append(f"🔐 예상 형량: {state['sentence_prediction']}")
    final_output = "\n".join([p for p in parts if p])
    return {"final_answer": final_output}                   # ✅

In [25]:
# # 워크플로우 langgraph 버전

# from langgraph.graph import StateGraph
# from langchain_core.runnables import RunnableLambda

# def create_workflow() -> StateGraph:
#     workflow = StateGraph(LegalCaseState)

#     # ── 1️⃣ 공통 노드 ────────────────────────────────────────
#     workflow.add_node("ExtractLegalIssue",  RunnableLambda(extract_legal_issue_node))
#     workflow.add_node("ExtractBasicFacts",  RunnableLambda(extract_basic_facts_node))
#     workflow.add_node("ClassifyCaseType",   RunnableLambda(classify_legal_domains_node))

#     # 최종 응답 노드
#     workflow.add_node("GenerateFinalAnswer", RunnableLambda(generate_final_answer_node))

#     # ── 2️⃣ 사건 유형별 브랜치 세트 ───────────────────────────
#     branch_labels = ["민사", "형사", "행정"]
#     for label in branch_labels:
#         gq   = f"GeneratePrecedentQuery_{label}"
#         sum_ = f"SummarizePrecedents_{label}"
#         law  = f"RecommendLaw_{label}"
#         sim  = f"SimulateJudgment_{label}"
#         sen  = f"PredictSentence_{label}"   # 형사에만 사용

#         workflow.add_node(gq,  RunnableLambda(lambda s, l=label: generate_precedent_queries_node({**s, "case_type": l})))
#         workflow.add_node(sum_, RunnableLambda(lambda s, l=label: summarize_precedents_node({**s, "case_type": l})))
#         workflow.add_node(law, RunnableLambda(lambda s, l=label: recommend_law_node({**s, "case_type": l})))
#         workflow.add_node(sim, RunnableLambda(lambda s, l=label: simulate_judgment_node({**s, "case_type": l})))
#         if label == "형사":
#             workflow.add_node(sen, RunnableLambda(lambda s, l=label: predict_sentence_node({**s, "case_type": l})))

#         # 순차 엣지 (브랜치 내부)
#         workflow.add_edge(gq,  sum_)
#         workflow.add_edge(sum_, law)
#         workflow.add_edge(law, sim)
#         if label == "형사":
#             workflow.add_edge(sim, sen)
#             workflow.add_edge(sen, "GenerateFinalAnswer")
#         else:
#             workflow.add_edge(sim, "GenerateFinalAnswer")

#     # ── 3️⃣ 사건 유형 라우팅 ─────────────────────────────────
#     def case_router(state: LegalCaseState) -> str:
#         """case_categories 중 첫 번째 일치 라벨 반환, 없으면 '기타'."""
#         cats = state.get("case_categories") or []
#         for l in ("형사", "민사", "행정"):
#             if l in cats:
#                 return l
#         return "기타"

#     workflow.add_conditional_edges(
#         "ClassifyCaseType",
#         case_router,
#         {
#             "민사":  "GeneratePrecedentQuery_민사",
#             "형사":  "GeneratePrecedentQuery_형사",
#             "행정":  "GeneratePrecedentQuery_행정",
#             "기타":  "GenerateFinalAnswer",        # fallback
#         }
#     )

#     # ── 4️⃣ 직렬 기본 흐름 (누락됐던 엣지 복구) ───────────────
#     workflow.set_entry_point("ExtractLegalIssue")
#     workflow.add_edge("ExtractLegalIssue", "ExtractBasicFacts")   # ✅ 추가
#     workflow.add_edge("ExtractBasicFacts", "ClassifyCaseType")    # ✅ 추가

#     # ── 5️⃣ 종료 지점 ───────────────────────────────────────
#     workflow.set_finish_point("GenerateFinalAnswer")

#     return workflow.compile()

# # ── 그래프 인스턴스 ─────────────────────────────────────────
# graph = create_workflow()

In [26]:
#워크플로 langfuse 버전

from langgraph.graph import StateGraph
from langchain_core.runnables import RunnableLambda
from langfuse.callback import CallbackHandler as LangfuseCallbackHandler
import os

# Langfuse 핸들러 설정
langfuse_handler = LangfuseCallbackHandler(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com"),
)

def create_workflow() -> StateGraph:
    workflow = StateGraph(LegalCaseState)

    # ── 1️⃣ 공통 노드 ───────────────────────────
    workflow.add_node("ExtractLegalIssue",  RunnableLambda(extract_legal_issue_node))
    workflow.add_node("ExtractBasicFacts",  RunnableLambda(extract_basic_facts_node))
    workflow.add_node("ClassifyCaseType",   RunnableLambda(classify_legal_domains_node))
    workflow.add_node("GenerateFinalAnswer", RunnableLambda(generate_final_answer_node))

    # ── 2️⃣ 브랜치 노드 ────────────────────────
    branch_labels = ["민사", "형사", "행정"]
    for label in branch_labels:
        gq   = f"GeneratePrecedentQuery_{label}"
        sum_ = f"SummarizePrecedents_{label}"
        law  = f"RecommendLaw_{label}"
        sim  = f"SimulateJudgment_{label}"
        sen  = f"PredictSentence_{label}"

        workflow.add_node(gq,  RunnableLambda(lambda s, l=label: generate_precedent_queries_node({**s, "case_type": l})))
        workflow.add_node(sum_, RunnableLambda(lambda s, l=label: summarize_precedents_node({**s, "case_type": l})))
        workflow.add_node(law, RunnableLambda(lambda s, l=label: recommend_law_node({**s, "case_type": l})))
        workflow.add_node(sim, RunnableLambda(lambda s, l=label: simulate_judgment_node({**s, "case_type": l})))
        if label == "형사":
            workflow.add_node(sen, RunnableLambda(lambda s, l=label: predict_sentence_node({**s, "case_type": l})))

        workflow.add_edge(gq,  sum_)
        workflow.add_edge(sum_, law)
        workflow.add_edge(law, sim)
        if label == "형사":
            workflow.add_edge(sim, sen)
            workflow.add_edge(sen, "GenerateFinalAnswer")
        else:
            workflow.add_edge(sim, "GenerateFinalAnswer")

    # ── 3️⃣ 사건 분류 조건부 흐름 ────────────────
    def case_router(state: LegalCaseState) -> str:
        cats = state.get("case_categories") or []
        for l in ("형사", "민사", "행정"):
            if l in cats:
                return l
        return "기타"

    workflow.add_conditional_edges(
        "ClassifyCaseType",
        case_router,
        {
            "민사":  "GeneratePrecedentQuery_민사",
            "형사":  "GeneratePrecedentQuery_형사",
            "행정":  "GeneratePrecedentQuery_행정",
            "기타":  "GenerateFinalAnswer",
        }
    )

    # ── 4️⃣ 기본 직렬 연결 ──────────────────────
    workflow.set_entry_point("ExtractLegalIssue")
    workflow.add_edge("ExtractLegalIssue", "ExtractBasicFacts")
    workflow.add_edge("ExtractBasicFacts", "ClassifyCaseType")
    workflow.set_finish_point("GenerateFinalAnswer")

    # ✅ Langfuse 트래킹 적용
    return workflow.compile().with_config({
        "callbacks": [langfuse_handler]
    })

# 그래프 인스턴스 생성
graph = create_workflow()

In [51]:
# 초기 상태 정의 (LegalCaseState 기준)
state = {
    "user_input": """
저는 2022년 3월 1일부터 서울시 마포구에 있는 다세대주택의 2층을 전세로 임차하여 거주해 왔습니다.  
계약 당시 보증금은 1억 5천만 원이었고, 계약기간은 2년으로 설정되어 있었으며, 2024년 2월 29일에 종료되었습니다.  
집주인과는 표준임대차계약서를 작성하였고, 보증금 반환과 관련하여 특약사항은 따로 명시하지 않았습니다.  
계약 갱신은 하지 않기로 하고, 저는 계약 종료일에 맞춰 이사를 준비하고 새로운 거처도 마련하였습니다.  

하지만 이사 하루 전인 2024년 2월 28일, 집주인이 갑자기 보증금이 당장 마련되지 않았다며 반환을 미룰 수밖에 없다고 통보해 왔습니다.  
이에 따라 저는 일단 새 집으로 이사를 완료한 후, 보증금 반환을 요청하는 내용증명을 보냈지만,  
집주인은 계속해서 “자금 사정이 어렵다”는 이유로 반환을 미루고 있습니다.  

현재 해당 주택에는 새로운 세입자도 들어오지 않은 상태이며,  
집주인은 전세보증금을 반환할 계획도, 일정도 명확하게 제시하지 않고 있습니다.  
저는 해당 보증금으로 새 집 전세자금을 충당해야 하는 상황이었기에  
현재 은행 대출을 받아 이사비용을 충당한 상태이며, 이로 인해 경제적 손해와 정신적 스트레스가 상당합니다.  

또한 집주인은 연락을 회피하고 있으며, 전화나 메시지에도 제대로 응답하지 않고 있어 자력으로 보증금을 돌려받기 어려운 상황입니다.  
이러한 상황에서 제가 취할 수 있는 법적 조치에는 어떤 것들이 있으며,  
실제로 소송을 진행하게 된다면 어떤 절차와 증거가 필요한지,  
보증금을 돌려받기까지 얼마나 걸릴 수 있는지 등 구체적인 조언을 받고 싶습니다.
"""
}

# LangGraph 실행
result = graph.invoke(state)

# 최종 답변 출력
print(result["final_answer"])

✅ 사건 분야: 민사
🔎 기초 사실: 1. 원고는 2022년 3월 1일부터 피고 소유의 서울시 마포구 소재 다세대주택 2층을 전세보증금 1억 5천만 원에 임차하였다.; 2. 계약기간은 2년으로 2024년 2월 29일까지로 정해졌다.; 3. 원고와 피고는 표준임대차계약서를 작성하였으며 보증금 반환에 관한 특약사항은 명시되지 않았다.; 4. 원고는 계약 종료일에 맞추어 이사를 준비하였으며 새로운 거처도 마련하였다.; 5. 피고는 이사 하루 전인 2024년 2월 28일, 보증금 마련이 되지 않아 반환을 미룰 수밖에 없다고 통보하였다.; 6. 원고는 이사 후 보증금 반환 요청을 내용증명으로 피고에게 발송하였다.; 7. 피고는 계속적으로 자금 사정을 이유로 보증금 반환을 미루고 있다.; 8. 해당 주택에는 새로운 세입자가 들어오지 않은 상태이다.; 9. 피고는 보증금 반환 계획 및 일정을 명확하게 제시하지 않고 있다.; 10. 원고는 은행 대출로 새 집 전세자금을 충당하였다.; 11. 피고는 연락을 회피하며 전화나 메시지에 제대로 응답하지 않고 있다.
⚖️ 법적 쟁점: 1. 임대차 계약 종료 후 임대인의 보증금 반환의무 불이행에 대한 법적 책임 

2. 임차인의 보증금 반환청구권 행사 절차: 내용증명 발송, 소송 제기 필요성 

3. 임대인의 연락 회피 및 부당한 지연에 따른 채무불이행 및 손해배상 청구 가능성 

4. 보증금 반환 지연에 따른 지연손해금 청구 범위 및 산정 방법 

5. 임차인이 은행 대출을 통해 이사비용을 충당함에 따라 발생한 추가 손해 및 이에 대한 배상 청구 가능성 

6. 계약 해지 후 임차인이 요구할 수 있는 강제집행을 위한 보전처분, 가압류 등 신청 필요성 

7. 주택임대차보호법에 근거한 임차인 보호 조항 및 이에 따른 효력 확인 

8. 임대인이 보증금을 반환하지 못하는 상황에서의 소송 준비: 계약서, 내용증명, 대출 증빙 등 증거 수집 

9. 임대차보증금 반환청구 소송에서의 절차적 진행: 소송 제기, 법원 판결, 강제집행 명령 신청 

In [ ]:
result = graph.invoke(state)
print(result.keys())          # 어떤 키가 내려왔는지 먼저 확인

dict_keys(['user_input', 'legal_issue', 'precedent_queries', 'precedent_summary', 'law_recommendation', 'legal_judgment_prediction', 'basic_facts', 'case_categories', 'final_answer'])
